# Data Information:

## Input variables:


## Output variable (desired target):


# Data preprocessing 

Import libraries and data, select a small sample set for trainning


In [108]:
from sklearn import datasets
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt

lol_origin = pd.read_csv('./leagueoflegends/leagueoflegends.csv')
lol = lol_origin.copy()
lol.tail()

,bu,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
7615,TCL,2018,Spring,Season,YC,0,1,SUP,34,"[0, 0, -18, -95, 45, -87, -117, 199, 126, 92, ...",...,Veigar,"[500, 500, 632, 982, 1258, 1667, 2060, 2390, 2...",Zeitnot,Xayah,"[500, 500, 564, 1027, 1320, 1727, 1963, 2276, ...",SnowFlower,Rakan,"[500, 500, 566, 801, 1004, 1234, 1463, 1674, 1...","['Ornn', 'Kalista', 'Ryze', 'Alistar', 'Shen']",https://matchhistory.euw.leagueoflegends.com/e...
7616,TCL,2018,Spring,Season,GAL,0,1,DP,39,"[0, 0, -86, -39, -207, -349, -60, -140, 187, -...",...,Ryze,"[500, 500, 612, 1000, 1193, 1626, 1979, 2318, ...",Ruvelius,Caitlyn,"[500, 500, 621, 977, 1420, 1809, 2047, 2413, 2...",Japone,Braum,"[500, 500, 587, 790, 1107, 1335, 1472, 1616, 1...","['Zoe', 'KogMaw', 'JarvanIV', 'Ornn', 'Rengar']",https://matchhistory.euw.leagueoflegends.com/e...
7617,OPL,2018,Spring,Season,SIN,0,1,DW,24,"[0, -8, -6, 116, 103, -92, -470, -958, -1998, ...",...,Malzahar,"[500, 500, 615, 903, 1225, 1617, 1976, 2304, 2...",k1ng,Ezreal,"[500, 508, 592, 986, 1400, 1753, 2196, 2538, 3...",Cupcake,Taric,"[500, 500, 585, 770, 976, 1222, 1446, 1627, 19...","['Azir', 'Kalista', 'Braum', 'Maokai', 'Alistar']",https://matchhistory.na.leagueoflegends.com/en...
7618,OPL,2018,Spring,Season,LGC,1,0,TTC,35,"[0, 0, -97, 33, 351, 284, 299, 263, 403, 623, ...",...,Veigar,"[500, 500, 561, 931, 1223, 1631, 1987, 2315, 2...",Low,Sivir,"[500, 500, 634, 930, 1304, 1691, 2042, 2384, 2...",Tilting,Janna,"[500, 500, 592, 788, 1006, 1251, 1490, 1704, 1...","['Kalista', 'Galio', 'Gnar', 'Jayce', 'Sejuani']",https://matchhistory.na.leagueoflegends.com/en...
7619,OPL,2018,Spring,Season,TTC,0,1,LGC,42,"[0, 0, -8, -225, -36, 73, 464, 184, 1171, 1409...",...,Malzahar,"[500, 500, 598, 968, 1165, 1477, 1801, 2016, 2...",Raid,Vayne,"[500, 500, 587, 954, 1327, 1749, 2007, 2471, 2...",Decoy,Braum,"[500, 500, 585, 793, 998, 1201, 1373, 1619, 18...","['KogMaw', 'Illaoi', 'Azir', 'Karma', 'Sejuani']",https://matchhistory.na.leagueoflegends.com/en...


In [109]:
#lol = lol_origin.drop(['goldblue','goldred','Address','bBarons','rBarons','redBans','blueBans',],1) 
#this is the way we treat 'player' as nominal variables

lol = lol_origin.drop(['goldblue','goldred','Address','bBarons','rBarons','redBans','blueBans','blueTop','blueMiddle','blueJungle','blueADC','blueSupport',
          'redTop','redMiddle','redJungle','redADC','redSupport'],1) 
# this is the way we treat 'player' as ordinal variables, therefore, we set them as dummies later

for x in ['golddiff','goldblueTop','goldblueJungle','goldblueMiddle','goldblueADC','goldblueSupport',
          'goldredTop','goldredJungle','goldredMiddle','goldredADC','goldredSupport']:
    lol[x] = [int(y.split(',')[14]) for y in lol_origin[x]]

for x in ['Top','Jungle','Middle','ADC','Support']:
    lol['golddiff' + x ] = lol['goldblue' + x ] - lol['goldred' + x ]
    del lol['goldblue' + x ]
    del lol['goldred' + x ]

In [110]:
def get_data15(feature):
    info = [x[2:-2] for x in feature]
    data15 = []
    for y in info:
        y = y.split('], [')
        j = 0
        for z in y:
            if z.strip() =='':
                break
            else:
                z=z.split(',')
                n =float(z[0])
                if n < 15.0 :
                    j = j + 1
        data15.append(j)
    return data15

In [111]:
for x in ['bKills','bTowers','bInhibs','bDragons','bHeralds',
          'rKills','rTowers','rInhibs','rDragons','rHeralds']:
    del lol[x]
    lol[x] = get_data15(lol_origin[x])

In [112]:
champ_names = lol_origin['blueTopChamp']
player_names = lol_origin['blueTop']
team_names = lol_origin['blueTeamTag']
for x in ['blueMiddleChamp','blueJungleChamp','blueADCChamp','blueSupportChamp',
          'redTopChamp','redMiddleChamp','redJungleChamp','redADCChamp','redSupportChamp']:
    champ_names = champ_names.append(lol_origin[x])
for x in ['blueMiddle','blueJungle','blueADC','blueSupport',
          'redTop','redMiddle','redJungle','redADC','redSupport']:
    player_names = player_names.append(lol_origin[x])
team_names = team_names.append(lol_origin['redTeamTag'])

### Processing the ordinal and nominal data,seperately

In [113]:
from sklearn.preprocessing import LabelEncoder

def nominal_transform(names,sets):
    le = LabelEncoder()
    le.fit_transform(names.astype(str))
    for x in sets:
        lol[x] = le.transform(lol_origin[x].astype(str))

In [114]:
Position = ['blueTop','blueMiddle','blueJungle','blueADC','blueSupport',
          'redTop','redMiddle','redJungle','redADC','redSupport']
Champ = [x+'Champ' for x in Position]
Team = ['blueTeamTag','redTeamTag']
Other = ['bu','Season','Type']

nominal_transform(champ_names,Champ)
#nominal_transform(player_names,Position)   this is the way we treat 'player' as nominal variables
nominal_transform(team_names,Team)

'''
champ_le = LabelEncoder()
champ_le.fit_transform(champ_names.astype(str))
for x in Champ:
    lol[x] = champ_le.transform(lol_origin[x].astype(str))

player_le = LabelEncoder()
player_le.fit_transform(player_names.astype(str))
for x in position:
    lol[x] = player_le.transform(lol_origin[x].astype(str))

team_le = LabelEncoder()
team_le.fit_transform(team_names.astype(str))
for x in ['blueTeamTag','redTeamTag']:
    lol[x] = team_le.transform(lol_origin[x].astype(str))
'''

for x in Other:
    le = LabelEncoder()
    lol[x] = le.fit_transform(lol_origin[x].astype(str))

In [115]:
dummies_Player = pd.get_dummies(lol_origin[['blueTop','blueMiddle','blueJungle','blueADC','blueSupport',
          'redTop','redMiddle','redJungle','redADC','redSupport']])

In [ ]:
Champ_list = sorted(champ_names.unique())
lol_origin.redBans = lol_origin.redBans.str.strip('[]').str.replace("'",'')
lol_origin.blueBans = lol_origin.blueBans.str.strip('[]').str.replace("'",'')
dummies_Champ_Bans = pd.DataFrame(np.zeros((len(lol_origin),len(Champ_list))),columns=Champ_list)
for i, Champ in enumerate(lol_origin.redBans):
    Champ = str(Champ)
    for ban in Champ.split(', '):
        dummies_Champ_Bans.ix[i, ban] = 1
for i, Champ in enumerate(lol_origin.blueBans):
    Champ = str(Champ)
    for ban in Champ.split(', '):
        dummies_Champ_Bans.ix[i, ban] = 1
del dummies_Champ_Bans['']
lol = lol.join(dummies_Champ_Bans)
lol = lol.join(dummies_Player)
print(len(lol.columns))

D:\python3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


3538


In [ ]:
y = lol['bResult']
X = lol.drop(['bResult','rResult'],1)

In [ ]:
labels = 'BlueWin', 'RedWin'
sizes = [y.value_counts()[1],y.value_counts()[0]]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

### Splitting data into 70% training and 30% test data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

### Bringing features onto the same scale(Standardization)

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train = stdsc.fit_transform(X_train)
X_test = stdsc.transform(X_test)

# Training Data with Different Models

In [ ]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(penalty='l1',n_iter=1, eta0=1, random_state=1)
ppn.fit(X_train, y_train)

print('accuracy of trainning = ' ,ppn.score(X_train,y_train))
print('accuracy of testing = ' ,ppn.score(X_test,y_test))

In [58]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l1',C=1,random_state=1)   
lr.fit(X_train, y_train)
print('accuracy of trainning = ' ,lr.score(X_train,y_train))
print('accuracy of testing = ' ,lr.score(X_test,y_test))

accuracy of trainning =  0.883202099738
accuracy of testing =  0.688976377953


<br>
<br>